In [ ]:
using Distributed
nprocs()

In [2]:
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [3]:
nprocs()

5

In [4]:
#= @everywhere using Pkg
@everywhere Pkg.add("DrWatson")=#

      From worker 2:	   Updating registry at `~/.julia/registries/General`
      From worker 2:	   Updating git-repo `https://github.com/JuliaRegistries/General.git`
      From worker 4:	   Updating registry at `~/.julia/registries/General`
      From worker 4:	   Updating git-repo `https://github.com/JuliaRegistries/General.git`
      From worker 3:	   Updating registry at `~/.julia/registries/General`
      From worker 3:	   Updating git-repo `https://github.com/JuliaRegistries/General.git`
      From worker 5:	   Updating registry at `~/.julia/registries/General`
      From worker 5:	   Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Resolving package versions...[>                                        ]  0.0 %
  Resolving package versions...[>                                        ]  0.0 %
  Resolving package versions...[>                                        ]  0.0 %
      From worker 2:	  Resolving package versions...
      From worker 5:	   Updating `~/.

CompositeException: TaskFailedException:
`Hwloc=0e44f5e4-bd66-52a0-8798-143a42290a1d` depends on `BinaryProvider`, but no such entry exists in the manifest.
Stacktrace:
 [1] pkgerror(::String, ::Vararg{String,N} where N) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Pkg/src/Types.jl:53
 [2] normalize_deps(::String, ::Base.UUID, ::Array{String,1}, ::Dict{String,Array{Pkg.Types.Stage1,1}}) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Pkg/src/manifest.jl:91
 [3] validate_manifest(::Dict{String,Array{Pkg.Types.Stage1,1}}) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Pkg/src/manifest.jl:105
 [4] Dict{Base.UUID,Pkg.Types.PackageEntry}(::Dict{String,Any}) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Pkg/src/manifest.jl:154
 [5] read_manifest(::String) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Pkg/src/manifest.jl:177
 [6] Pkg.Types.EnvCache(::Nothing) at

In [35]:
#=@everywhere using Pkg
@everywhere Pkg.add("SharedArrays")=#

      From worker 6:	  Resolving package versions...


  Resolving package versions...


      From worker 4:	  Resolving package versions...
      From worker 2:	  Resolving package versions...


   Updating `~/Repos/advection-diffusion-catalysis/Project.toml`


      From worker 5:	  Resolving package versions...
      From worker 3:	  Resolving package versions...


  [1a1011a3] + SharedArrays 
   Updating `~/Repos/advection-diffusion-catalysis/Manifest.toml`
 [no changes]


      From worker 6:	   Updating `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 4:	   Updating `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 6:	  [1a1011a3] + SharedArrays 
      From worker 6:	   Updating `~/Repos/advection-diffusion-catalysis/Manifest.toml`
      From worker 6:	 [no changes]
      From worker 4:	  [1a1011a3] + SharedArrays 
      From worker 4:	   Updating `~/Repos/advection-diffusion-catalysis/Manifest.toml`
      From worker 4:	 [no changes]
      From worker 2:	   Updating `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 5:	   Updating `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 3:	   Updating `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 2:	  [1a1011a3] + SharedArrays 
      From worker 2:	   Updating `~/Repos/advection-diffusion-catalysis/Manifest.toml`
      From worker 2:	 [no changes]
      From worker 5:	  [1a1011a3] + SharedArrays 
    

In [6]:
@everywhere using DrWatson
@everywhere @quickactivate :Catalyst
@everywhere using BenchmarkTools

@everywhere function benchmark_distributed_evaluation(parameters,N)
    errors = zeros(N) # init model errors
    # we need to read the experiment data in this block
    input_exp = Float64[]
    output_exp = Float64[]
    for row in CSV.File(datadir("experiment/new-data.csv"); delim = " ")
         push!(input_exp, row.I)
         push!(output_exp, row.O)
    end
    # done reading 
    
    F = Array{Future}(undef,N)
    
    for i in 1:N 
        F[i] = @spawnat i+1 (Catalyst.solve(parameters[1], parameters[2], 1.,
                                      input_exp, output_exp, progress=false, 
                                      microcomp_type=:nonlinear,
                                      Q=parameters[3], kₙ=parameters[4],
                                      calibration=true))
        end 
    
    for i in 1:N
    errors[i] = fetch(F[i])
    end
    return errors
end

parameters = [1e-4, 1., 1., 10.]
@time benchmark_distributed_evaluation(parameters,2)

┌ Info: Precompiling Catalyst [01e5440a-74e3-4b45-b8b9-c13ddfc5051b]
└ @ Base loading.jl:1260
ERROR: LoadError: LoadError: 

      From worker 4:	ERROR: LoadError: LoadError: ArgumentError: Package ForwardDiff [f6369f11-7733-5829-9624-2563aa707210] is required but does not seem to be installed:
      From worker 4:	 - Run `Pkg.instantiate()` to install all recorded dependencies.
      From worker 4:	
      From worker 2:	ERROR: LoadError: LoadError: ArgumentError: Package ForwardDiff [f6369f11-7733-5829-9624-2563aa707210] is required but does not seem to be installed:
      From worker 2:	 - Run `Pkg.instantiate()` to install all recorded dependencies.
      From worker 2:	
      From worker 3:	ERROR: LoadError: LoadError: ArgumentError: Package ForwardDiff [f6369f11-7733-5829-9624-2563aa707210] is required but does not seem to be installed:
      From worker 3:	 - Run `Pkg.instantiate()` to install all recorded dependencies.
      From worker 3:	
      From worker 4:	Stacktrace:
      From worker 2:	Stacktrace:
      From worker 3:	Stacktrace:
      From worker 5:	ERROR: LoadError: LoadError: ArgumentError:

ArgumentError: Package ForwardDiff [f6369f11-7733-5829-9624-2563aa707210] is required but does not seem to be installed:
 - Run `Pkg.instantiate()` to install all recorded dependencies.

Stacktrace:

      From worker 4:	 [1] _require(::Base.PkgId) at ./loading.jl:998
      From worker 4:	 [2] require(::Base.PkgId) at ./loading.jl:927
      From worker 4:	 [3] require(::Module, ::Symbol) at ./loading.jl:922
      From worker 4:	 [4] include(::Module, ::String) at ./Base.jl:377
      From worker 4:	 [5] include(::String) at /home/amjalled/.julia/packages/Tensors/jWUZM/src/Tensors.jl:3
      From worker 4:	 [6] top-level scope at /home/amjalled/.julia/packages/Tensors/jWUZM/src/Tensors.jl:160
      From worker 4:	 [7] include(::Module, ::String) at ./Base.jl:377
      From worker 4:	 [8] top-level scope at none:2
      From worker 4:	 [9] eval at ./boot.jl:331 [inlined]
      From worker 4:	 [10] eval(::Expr) at ./client.jl:449
      From worker 4:	 [11] top-level scope at ./none:3
      From worker 4:	in expression starting at /home/amjalled/.julia/packages/Tensors/jWUZM/src/automatic_differentiation.jl:1
      From worker 4:	in expression starting at /home/amjalled/.julia/packages/


 [1] _require(

...

      From worker 2:	 [1] error(::String) at ./error.jl:33
      From worker 2:	 [2] compilecache(::Base.PkgId, ::String) at ./loading.jl:1272
      From worker 2:	 [3] _require(::Base.PkgId) at ./loading.jl:1029
      From worker 2:	 [4] require(::Base.PkgId) at ./loading.jl:927
      From worker 2:	 [5] require(::Module, ::Symbol) at ./loading.jl:922
      From worker 2:	 [6] include(::Module, ::String) at ./Base.jl:377
      From worker 2:	 [7] top-level scope at none:2
      From worker 2:	 [8] eval at ./boot.jl:331 [inlined]
      From worker 2:	 [9] eval(::Expr) at ./client.jl:449
      From worker 2:	 [10] top-level scope at ./none:3
      From worker 2:	in expression starting at /home/amjalled/Repos/advection-diffusion-catalysis/src/Catalyst.jl:4
      From worker 3:	 [1] error(::String) at ./error.jl:33
      From worker 3:	 [2] compilecache(::Base.PkgId, ::String) at ./loading.jl:1272
      From worker 3:	 [3] _require(::Base.PkgId) at ./loading.jl:1029
      From worker 3:	 

::

      From worker 5:	 [1] error(::String) at ./error.jl:33
      From worker 5:	 [2] compilecache(::Base.PkgId, ::String) at ./loading.jl:1272
      From worker 5:	 [3] _require(::Base.PkgId) at ./loading.jl:1029
      From worker 5:	 [4] require(::Base.PkgId) at ./loading.jl:927
      From worker 5:	 [5] require(::Module, ::Symbol) at ./loading.jl:922
      From worker 5:	 [6] include(::Module, ::String) at ./Base.jl:377
      From worker 5:	 [7] top-level scope at none:2
      From worker 5:	 [8] eval at ./boot.jl:331 [inlined]
      From worker 5:	 [9] eval(::Expr) at ./client.jl:449
      From worker 5:	 [10] top-level scope at ./none:3
      From worker 5:	in expression starting at /home/amjalled/Repos/advection-diffusion-catalysis/src/Catalyst.jl:4


String) at ./error.jl:33
 [2] compilecache(::Base.PkgId, ::String) at ./loading.jl:1272
 [3] _require(::Base.PkgId) at ./loading.jl:1029
 [4] require(::Base.PkgId) at ./loading.jl:927
 [5] require(::Module, ::Symbol) at ./loading.jl:922
 [6] include(::Module, ::String) at ./Base.jl:377
 [7] top-level scope at none:2
 [8] eval at ./boot.jl:331 [inlined]
 [9] eval(::Expr) at ./client.jl:449
 [10] top-level scope at ./none:3
in expression starting at /home/amjalled/Repos/advection-diffusion-catalysis/src/Catalyst.jl:4


CompositeException: On worker 2:
Failed to precompile Catalyst [01e5440a-74e3-4b45-b8b9-c13ddfc5051b] to /home/amjalled/.julia/compiled/v1.4/Catalyst/J9Gan_O9VyG.ji.
error at ./error.jl:33
compilecache at ./loading.jl:1272
_require at ./loading.jl:1029
require at ./loading.jl:927
require at ./loading.jl:922
top-level scope at /home/amjalled/.julia/packages/DrWatson/PGLXY/src/project_setup.jl:180
eval at ./boot.jl:331
#101 at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Distributed/src/process_messages.jl:290
run_work_thunk at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Distributed/src/process_messages.jl:79
run_work_thunk at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Distributed/src/process_messages.jl:88
#94 at ./task.jl:358

...and 4 more exception(s).
